## Converts png to PPM

In [29]:
#@title Converts png to PPM
from PIL import Image
def convert_ppm3(filename, tofilename):

    im = Image.open(filename) # Can be many different formats.
    pix = im.load()
    print(im.size)  # Get the width and hight of the image for iterating over
    f = open(tofilename, "w")
    f.write("P3\n{} {}\n255\n".format(im.size[0],im.size[1]))
    for x in range(im.size[0]):
        for y in range(im.size[1]):
            rgba = pix[im.size[0]-y-1,x];
            f.write("{}\n{}\n{}\n".format(rgba[0],rgba[1],rgba[2]))
    f.close()

for i in range(10):
    filename = "{}.png".format(i)
    tofilename= "{}.ppm".format(i)
    convert_ppm3(filename, tofilename)


(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)
(600, 600)


In [11]:
im = Image.open(filename) # Can be many different formats.
pix = im.load()
print(im.size)  # Get the width and hight of the image for iterating over
f = open(tofilename, "w")
f.write("P3\n{}\n{}\n255\n".format(im.size[0],im.size[1]))
for x in range(im.size[0]):
    for y in range(im.size[1]):
        rgba = pix[y,x];
        f.write("{}\n{}\n{}\n".format(rgba[0],rgba[1],rgba[2]))
f.close()

## Neural Network Model

In [30]:
#@title 1. Setup
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2 as cv

# Seed
SEED = 111
np.random.seed(seed=SEED)
tf.random.set_seed(SEED)

In [31]:
#@title 2. Dense Model Binary data
x_train_resize = []
x_test_resize = []

num_classes = 10
input_shape = (10*10)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

for i,image in enumerate(x_train):
    image = cv.resize(image, (10,10))
    image = image > 0
    image = image.astype(dtype="int")
    x_train_resize.append(image)
for i, image in enumerate(x_test):
    image = cv.resize(image, (10,10))
    image = image > 0
    image = image.astype(dtype="int")
    x_test_resize.append(image)

x_train_resize = np.asarray(x_train_resize)
x_train_resize = x_train_resize.reshape(60000,10*10)
x_test_resize= np.asarray(x_test_resize)
x_test_resize = x_test_resize.reshape(10000,10*10)



11493376/11490434 [==============================] - 0s 0us/step


In [32]:
#@title 3. Dense Model Init
nnn = 10 #30
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(nnn,activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1010      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
Total params: 1,340
Trainable params: 1,340
Non-trainable params: 0
_________________________________________________________________


In [33]:
#@title 4. dense model compile
batch_size = 128
epochs = 30
op = keras.optimizers.SGD(learning_rate = 0.1, momentum = 0.9)
op = keras.optimizers.Adam(learning_rate = 0.05)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
#model.compile(loss="categorical_crossentropy", optimizer=op, metrics=["accuracy"])

model.fit(x_train_resize, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Epoch 1/30
422/422 [==============================] - 1s 2ms/step - loss: 1.6054 - accuracy: 0.4317 - val_loss: 0.8819 - val_accuracy: 0.7188
Epoch 2/30
422/422 [==============================] - 1s 2ms/step - loss: 0.7423 - accuracy: 0.7657 - val_loss: 0.5655 - val_accuracy: 0.8238
Epoch 3/30
422/422 [==============================] - 1s 2ms/step - loss: 0.5946 - accuracy: 0.8113 - val_loss: 0.5077 - val_accuracy: 0.8422
Epoch 4/30
422/422 [==============================] - 1s 2ms/step - loss: 0.5418 - accuracy: 0.8277 - val_loss: 0.4606 - val_accuracy: 0.8568
Epoch 5/30
422/422 [==============================] - 1s 2ms/step - loss: 0.5080 - accuracy: 0.8384 - val_loss: 0.4356 - val_accuracy: 0.8677
Epoch 6/30
422/422 [==============================] - 1s 2ms/step - loss: 0.4866 - accuracy: 0.8465 - val_loss: 0.4234 - val_accuracy: 0.8705
Epoch 7/30
422/422 [==============================] - 1s 2ms/step - loss: 0.4702 - accuracy: 0.8527 - val_loss: 0.4056 - val_accuracy: 0.8767
Epoch 

In [34]:
model.predict(np.array([x_train_resize[1000],]))

array([[9.9004656e-01, 1.3267862e-11, 2.6479608e-04, 7.5979609e-05,
        5.2724783e-05, 8.6805290e-03, 4.2114848e-06, 1.2275509e-05,
        1.2307552e-04, 7.3991937e-04]], dtype=float32)

In [ ]:
#@ title 5. Output the weights
#https://numpy.org/doc/stable/reference/generated/numpy.savetxt.html
np.savetxt('test0w.txt', model.layers[0].get_weights()[0], delimiter=',',newline=',\n')
np.savetxt('test0b.txt', model.layers[0].get_weights()[1], delimiter=',',newline=',\n')
np.savetxt('test1w.txt', model.layers[1].get_weights()[0], delimiter=',',newline=',\n')
np.savetxt('test1b.txt', model.layers[1].get_weights()[1], delimiter=',',newline=',\n')
np.savetxt('test2w.txt', model.layers[2].get_weights()[0], delimiter=',',newline=',\n')
np.savetxt('test2b.txt', model.layers[2].get_weights()[1], delimiter=',',newline=',\n')
np.savetxt('test3w.txt', model.layers[3].get_weights()[0], delimiter=',',newline=',\n')
np.savetxt('test3b.txt', model.layers[3].get_weights()[1], delimiter=',',newline=',\n')